## Import dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_recall_curve, auc, f1_score, confusion_matrix,
    classification_report, roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

## Load Datasets

In [2]:
try:
    fraud_data = pd.read_csv('../data/Fraud_Data_merged.csv')
    creditcard_data = pd.read_csv('../data/creditcard_cleaned.csv')

    if 'signup_time' in fraud_data.columns and not pd.api.types.is_datetime64_any_dtype(fraud_data['signup_time']):
        fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'], errors='coerce')
    if 'purchase_time' in fraud_data.columns and not pd.api.types.is_datetime64_any_dtype(fraud_data['purchase_time']):
        fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'], errors='coerce')

    initial_rows_fraud = fraud_data.shape[0]
    fraud_data.dropna(subset=['signup_time', 'purchase_time'], inplace=True)
    if fraud_data.shape[0] < initial_rows_fraud:
        print(f"Dropped {initial_rows_fraud - fraud_data.shape[0]} rows from Fraud_Data due to invalid signup_time or purchase_time after loading merged CSV.")
    
    print("Pre-processed datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Please ensure the specified CSV files are in the same directory as this script.")
    exit()


Pre-processed datasets loaded successfully.


## Define Features

In [3]:
X_fraud = fraud_data.drop(columns=[
    'user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address', 'class'
])
y_fraud = fraud_data['class']

X_creditcard = creditcard_data.drop(columns=['Time', 'Class'])
y_creditcard = creditcard_data['Class']

## Train-Test Split

In [4]:
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(
    X_fraud, y_fraud, test_size=0.3, random_state=42, stratify=y_fraud
)
X_creditcard_train, X_creditcard_test, y_creditcard_train, y_creditcard_test = train_test_split(
    X_creditcard, y_creditcard, test_size=0.3, random_state=42, stratify=y_creditcard
)

## Handle class imbalance

In [5]:
numerical_cols_fraud_train = X_fraud_train.select_dtypes(include=np.number).columns
categorical_cols_fraud_train = X_fraud_train.select_dtypes(include='object').columns

encoder_fraud_smote = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_fraud_train_cat_encoded = encoder_fraud_smote.fit_transform(X_fraud_train[categorical_cols_fraud_train])
X_fraud_train_cat_encoded_df = pd.DataFrame(X_fraud_train_cat_encoded, columns=encoder_fraud_smote.get_feature_names_out(categorical_cols_fraud_train), index=X_fraud_train.index)
X_fraud_train_combined = pd.concat([X_fraud_train[numerical_cols_fraud_train], X_fraud_train_cat_encoded_df], axis=1)

smote = SMOTE(random_state=42)
X_fraud_train_res, y_fraud_train_res = smote.fit_resample(X_fraud_train_combined, y_fraud_train)
print(f"Fraud_Data training set resampled shape (SMOTE): {Counter(y_fraud_train_res)}")

# Handle Class Imbalance (for creditcard.csv)
rus = RandomUnderSampler(random_state=42)
X_creditcard_train_res, y_creditcard_train_res = rus.fit_resample(X_creditcard_train, y_creditcard_train)
print(f"Creditcard_Data training set resampled shape (RandomUnderSampler): {Counter(y_creditcard_train_res)}")

Fraud_Data training set resampled shape (SMOTE): Counter({0: 95872, 1: 95872})
Creditcard_Data training set resampled shape (RandomUnderSampler): Counter({0: 331, 1: 331})


## Normalization and scaling

In [6]:
scaler_fraud = StandardScaler()
numerical_cols_fraud_resampled = X_fraud_train_res.columns # All columns are numerical after encoding
X_fraud_train_res[numerical_cols_fraud_resampled] = scaler_fraud.fit_transform(X_fraud_train_res[numerical_cols_fraud_resampled])

X_fraud_test_cat_encoded = encoder_fraud_smote.transform(X_fraud_test[categorical_cols_fraud_train])
X_fraud_test_cat_encoded_df = pd.DataFrame(X_fraud_test_cat_encoded, columns=encoder_fraud_smote.get_feature_names_out(categorical_cols_fraud_train), index=X_fraud_test.index)
X_fraud_test_combined = pd.concat([X_fraud_test[numerical_cols_fraud_train], X_fraud_test_cat_encoded_df], axis=1)
X_fraud_test = scaler_fraud.transform(X_fraud_test_combined)
X_fraud_test = pd.DataFrame(X_fraud_test, columns=X_fraud_train_res.columns, index=X_fraud_test_combined.index)


scaler_creditcard = StandardScaler()
numerical_cols_creditcard = X_creditcard_train_res.select_dtypes(include=np.number).columns
X_creditcard_train_res[numerical_cols_creditcard] = scaler_creditcard.fit_transform(X_creditcard_train_res[numerical_cols_creditcard])
X_creditcard_test[numerical_cols_creditcard] = scaler_creditcard.transform(X_creditcard_test[numerical_cols_creditcard])

## Model Training

In [ ]:
# Logistic Regression
lr_fraud = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')
lr_creditcard = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')

# Random Forest Classifier
rf_fraud = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')
rf_creditcard = RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced')

lr_fraud.fit(X_fraud_train_res, y_fraud_train_res)
y_pred_lr_fraud = lr_fraud.predict(X_fraud_test)
y_prob_lr_fraud = lr_fraud.predict_proba(X_fraud_test)[:, 1]

print("\nLogistic Regression (Fraud_Data) Evaluation:")
print("Confusion Matrix:\n", confusion_matrix(y_fraud_test, y_pred_lr_fraud))
print("Classification Report:\n", classification_report(y_fraud_test, y_pred_lr_fraud))
precision_lr_fraud, recall_lr_fraud, _ = precision_recall_curve(y_fraud_test, y_prob_lr_fraud)
auc_pr_lr_fraud = auc(recall_lr_fraud, precision_lr_fraud)
print(f"AUC-PR: {auc_pr_lr_fraud:.4f}")
print(f"F1-Score: {f1_score(y_fraud_test, y_pred_lr_fraud):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_fraud_test, y_prob_lr_fraud):.4f}")

print("\nTraining Random Forest for Fraud_Data...")
rf_fraud.fit(X_fraud_train_res, y_fraud_train_res)
y_pred_rf_fraud = rf_fraud.predict(X_fraud_test)
y_prob_rf_fraud = rf_fraud.predict_proba(X_fraud_test)[:, 1]

print("\nRandom Forest (Fraud_Data) Evaluation:")
print("Confusion Matrix:\n", confusion_matrix(y_fraud_test, y_pred_rf_fraud))
print("Classification Report:\n", classification_report(y_fraud_test, y_pred_rf_fraud))
precision_rf_fraud, recall_rf_fraud, _ = precision_recall_curve(y_fraud_test, y_prob_rf_fraud)
auc_pr_rf_fraud = auc(recall_rf_fraud, precision_rf_fraud)
print(f"AUC-PR: {auc_pr_rf_fraud:.4f}")
print(f"F1-Score: {f1_score(y_fraud_test, y_pred_rf_fraud):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_fraud_test, y_prob_rf_fraud):.4f}")


print("\n--- Training and Evaluating Models for Creditcard.csv ---")

# Logistic Regression for Creditcard_Data
print("\nTraining Logistic Regression for Creditcard_Data...")
lr_creditcard.fit(X_creditcard_train_res, y_creditcard_train_res)
y_pred_lr_creditcard = lr_creditcard.predict(X_creditcard_test)
y_prob_lr_creditcard = lr_creditcard.predict_proba(X_creditcard_test)[:, 1]

print("\nLogistic Regression (Creditcard_Data) Evaluation:")
print("Confusion Matrix:\n", confusion_matrix(y_creditcard_test, y_pred_lr_creditcard))
print("Classification Report:\n", classification_report(y_creditcard_test, y_pred_lr_creditcard))
precision_lr_creditcard, recall_lr_creditcard, _ = precision_recall_curve(y_creditcard_test, y_prob_lr_creditcard)
auc_pr_lr_creditcard = auc(recall_lr_creditcard, precision_lr_creditcard)
print(f"AUC-PR: {auc_pr_lr_creditcard:.4f}")
print(f"F1-Score: {f1_score(y_creditcard_test, y_pred_lr_creditcard):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_creditcard_test, y_prob_lr_creditcard):.4f}")


# Random Forest for Creditcard_Data
print("\nTraining Random Forest for Creditcard_Data...")
rf_creditcard.fit(X_creditcard_train_res, y_creditcard_train_res)
y_pred_rf_creditcard = rf_creditcard.predict(X_creditcard_test)
y_prob_rf_creditcard = rf_creditcard.predict_proba(X_creditcard_test)[:, 1]

print("\nRandom Forest (Creditcard_Data) Evaluation:")
print("Confusion Matrix:\n", confusion_matrix(y_creditcard_test, y_pred_rf_creditcard))
print("Classification Report:\n", classification_report(y_creditcard_test, y_pred_rf_creditcard))
precision_rf_creditcard, recall_rf_creditcard, _ = precision_recall_curve(y_creditcard_test, y_prob_rf_creditcard)
auc_pr_rf_creditcard = auc(recall_rf_creditcard, precision_rf_creditcard)
print(f"AUC-PR: {auc_pr_rf_creditcard:.4f}")
print(f"F1-Score: {f1_score(y_creditcard_test, y_pred_rf_creditcard):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_creditcard_test, y_prob_rf_creditcard):.4f}")



Logistic Regression (Fraud_Data) Evaluation:
Confusion Matrix:
 [[23486 17603]
 [ 2095  2150]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.57      0.70     41089
           1       0.11      0.51      0.18      4245

    accuracy                           0.57     45334
   macro avg       0.51      0.54      0.44     45334
weighted avg       0.84      0.57      0.66     45334

AUC-PR: 0.1192
F1-Score: 0.1792
ROC-AUC: 0.5560

Training Random Forest for Fraud_Data...

Random Forest (Fraud_Data) Evaluation:
Confusion Matrix:
 [[40710   379]
 [ 1972  2273]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97     41089
           1       0.86      0.54      0.66      4245

    accuracy                           0.95     45334
   macro avg       0.91      0.76      0.82     45334
weighted avg       0.94      0.95      0.94     45334

AUC-PR: 0.5588
F1-Scor

## Model Selection

In [9]:
print("\nFor Fraud_Data.csv (E-commerce):")
print(f"Logistic Regression AUC-PR: {auc_pr_lr_fraud:.4f}, F1-Score: {f1_score(y_fraud_test, y_pred_lr_fraud):.4f}")
print(f"Random Forest AUC-PR: {auc_pr_rf_fraud:.4f}, F1-Score: {f1_score(y_fraud_test, y_pred_rf_fraud):.4f}")
print("\nRandom Forest generally outperforms Logistic Regression on Fraud_Data.csv, especially in terms of AUC-PR and F1-Score, indicating better handling of the imbalanced nature and capturing more complex patterns. This is expected as ensemble methods are typically more powerful.")

print("\nFor Creditcard.csv (Bank Transactions):")
print(f"Logistic Regression AUC-PR: {auc_pr_lr_creditcard:.4f}, F1-Score: {f1_score(y_creditcard_test, y_pred_lr_creditcard):.4f}")
print(f"Random Forest AUC-PR: {auc_pr_rf_creditcard:.4f}, F1-Score: {f1_score(y_creditcard_test, y_pred_rf_creditcard):.4f}")


For Fraud_Data.csv (E-commerce):
Logistic Regression AUC-PR: 0.1192, F1-Score: 0.1792
Random Forest AUC-PR: 0.5588, F1-Score: 0.6591

Random Forest generally outperforms Logistic Regression on Fraud_Data.csv, especially in terms of AUC-PR and F1-Score, indicating better handling of the imbalanced nature and capturing more complex patterns. This is expected as ensemble methods are typically more powerful.

For Creditcard.csv (Bank Transactions):
Logistic Regression AUC-PR: 0.3401, F1-Score: 0.0868
Random Forest AUC-PR: 0.7395, F1-Score: 0.0889
